The Federal Reserve (FED) hiked interest rates by 0.25 percent on March 16th, 2022, and announced six more hikes through the end of the year to combat the greatest inflation in four decades.The FED's aggressive monetary strategy is not limited to interest rate hikes. The other side entails lowering its $9 trillion balance sheet by first ceasing to purchase securities, which had been launched to support the economy during the pandemic, and then beginning to sell a portion of it to reduce money in circulation.

An attempt is made in here is to forecast the evolution of the S&aP500 index for possible scenarios of the FED balance sheet evolution.

### 1. Historical Data Set
The first step to start the exercise consists in gathering the data of interest, namely:

S&P500 index historical values,
the FED balance sheet historical values,
the historical interest rates for the United States.
The S&P500 historical values can be downloaded easily from Yahoo! Finance using Python and the yahooquery package

In [31]:
from yahooquery import Ticker
import pandas as pd
sp500 = Ticker("^GSPC").history(period='21Y', interval='1d')
sp500 = sp500.reset_index()
sp500["date"] = pd.to_datetime(sp500["date"])
sp500.set_index("date",inplace=True)
sp500

,symbol,high,close,volume,open,low,adjclose
date,,,,,,,
2001-06-11,^GSPC,1264.959961,1254.390015,870100000,1264.959961,1249.229980,1254.390015
2001-06-12,^GSPC,1261.000000,1255.849976,1136500000,1254.390015,1235.750000,1255.849976
2001-06-13,^GSPC,1259.750000,1241.599976,1063600000,1255.849976,1241.589966,1241.599976
2001-06-14,^GSPC,1241.599976,1219.869995,1242900000,1241.599976,1218.900024,1219.869995
2001-06-15,^GSPC,1221.500000,1214.359985,1635550000,1219.869995,1203.030029,1214.359985
...,...,...,...,...,...,...,...
2022-06-02,^GSPC,4177.509766,4176.819824,3604930000,4095.409912,4074.370117,4176.819824
2022-06-03,^GSPC,4142.669922,4108.540039,3107080000,4137.569824,4098.669922,4108.540039
2022-06-06,^GSPC,4168.779785,4121.430176,3852050000,4134.720215,4109.180176,4121.430176


In [17]:
#fedProject.py
# . . .
data_codes  = {"Base: Total ($ Mil)": "BOGMBASE",
               "Base: Currency in Circulation ($ Mil)": "WCURCIR",
               # Assets
               "Balance Sheet: Total Assets ($ Mil)": "WALCL",
               "Balance Sheet Securities, Prem-Disc, Repos, and Loans ($ Mil)": "WSRLL",
               "Balance Sheet: Securities Held Outright ($ Mil)": "WSHOSHO",
               ### breakdown of securities holdings ###
               "Balance Sheet: U.S. Treasuries Held Outright ($ Mil)":"WSHOTSL",
               "Balance Sheet: Federal Agency Debt Securities ($ Mil)" : "WSHOFADSL",
               "Balance Sheet: Mortgage-Backed Securities ($ Mil)": "WSHOMCB",
               # other forms of lending
               "Balance Sheet: Repos ($ Mil)": "WORAL",
               "Balance Sheet: Central Bank Liquidity Swaps ($ Mil)" : "SWPT",
               "Balance Sheet: Direct Lending ($ Mil)" : "WLCFLL",
               # unamortized value of securities held (due to changes in interest rates)
               "Balance Sheet: Unamortized Security Premiums ($ Mil)": "WUPSHO",
               # Liabilities
               "Balance Sheet: Total Liabilities ($ Mil)" : "WLTLECL",
               "Balance Sheet: Federal Reserve Notes Outstanding ($ Mil)" : "WLFN",
               "Balance Sheet: Reverse Repos ($ Mil)": "WLRRAL",
               ### Major share of deposits 
               "Balance Sheet: Excess Reserves ($ Mil)": "EXCSRESNW",
               "Balance Sheet: Required Reserves ($ Mil)": "RESBALREQW",
               "Balance Sheet: Total Reserves ($ Mil)": "WRESBAL",
               "Balance Sheet: Deposits from Dep. Institutions ($ Mil)":"WLODLL",
               "Balance Sheet: U.S. Treasury General Account ($ Mil)": "WDTGAL",
               "Balance Sheet: Other Deposits ($ Mil)": "WOTHLB",
               "Balance Sheet: All Deposits ($ Mil)": "WLDLCL",
               # Interest Rates
               "Federal Funds Target (Pre-Crisis)": "DFEDTAR",
               "Federal Funds (Upper) Target":"DFEDTARU",
               "Effective Federal Funds Rate":"DFF",
               "Interest on Excess Reserves":"IOER",
               "Interest on Reserve Balances":"IORB",

               # Req Reserves and Vault Cash
               "Vault Cash ($ Mil)": "TLVAULTW",
               "Vault Cash Used as Req. ($ Mil)": "VAULT",
               }
               
# Select start and end dates
start = datetime.datetime(2001, 6, 11)
end = datetime.datetime.today()

# freq refers to data frequency. Choose "D", "W", "M", "Q", "A"
# a number may also be place in front of a letter. "2D" indicates
#       alternating days
data = gather_data(data_codes = data_codes, start = start, 
                   end = end, freq = "W")
data["Interest on Reserves"] = data["Interest on Excess Reserves"].fillna(0).add(data["Interest on Reserve Balances"].fillna(0))
bil_to_mil_list  = ["Balance Sheet: Total Reserves ($ Mil)",
                    "Balance Sheet: Other Deposits ($ Mil)",
                    "Base: Currency in Circulation ($ Mil)"]
for key in bil_to_mil_list:
    data[key] = bil_to_mil(data[key])

In [18]:
#FRED.py
#. . . 
def bil_to_mil(series):
    return series* 10**3

In [19]:
#FRED.py
import pandas as pd
import pandas_datareader.data as web
import datetime

def gather_data(data_codes, 
                start, 
                end = datetime.datetime.today(), 
                freq = "M"):
    i = 0
    # dct.items() calls key and value that key points to
    for key, val in data_codes.items():
        if i == 0:
            # Create dataframe for first variable, then rename column
            df = web.DataReader(
                val, "fred", start, end).resample(freq).mean()
            df.rename(columns = {val:key}, inplace = True) 
            # setting i to None will cause the next block of code to execute,
            # placing data within df instead of creating a new dataframe for
            # each variable
            i = None
        else:
            # If dataframe already exists, add new column
            df[key] = web.DataReader(val, "fred", start, end).resample(freq).mean()

    return df

In [20]:
data

,Base: Total ($ Mil),Base: Currency in Circulation ($ Mil),Balance Sheet: Total Assets ($ Mil),"Balance Sheet Securities, Prem-Disc, Repos, and Loans ($ Mil)",Balance Sheet: Securities Held Outright ($ Mil),Balance Sheet: U.S. Treasuries Held Outright ($ Mil),Balance Sheet: Federal Agency Debt Securities ($ Mil),Balance Sheet: Mortgage-Backed Securities ($ Mil),Balance Sheet: Repos ($ Mil),Balance Sheet: Central Bank Liquidity Swaps ($ Mil),...,Balance Sheet: Other Deposits ($ Mil),Balance Sheet: All Deposits ($ Mil),Federal Funds Target (Pre-Crisis),Federal Funds (Upper) Target,Effective Federal Funds Rate,Interest on Excess Reserves,Interest on Reserve Balances,Vault Cash ($ Mil),Vault Cash Used as Req. ($ Mil),Interest on Reserves
DATE,,,,,,,,,,,,,,,,,,,,,
2001-07-01,616000.0,593979.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,279.0,NaN,3.821429,NaN,3.872857,NaN,NaN,43852.0,31.644,0.000000
2001-07-08,NaN,598386.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,285.0,NaN,3.750000,NaN,3.715714,NaN,NaN,44253.0,NaN,0.000000
2001-07-15,NaN,601487.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,283.0,NaN,3.750000,NaN,3.741429,NaN,NaN,44253.0,NaN,0.000000
2001-07-22,NaN,601089.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,325.0,NaN,3.750000,NaN,3.775714,NaN,NaN,43365.0,NaN,0.000000
2001-07-29,NaN,600585.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,303.0,NaN,3.750000,NaN,3.807143,NaN,NaN,43365.0,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-06,6134500.0,2242537.0,8904455.0,8798381.0,8442811.0,5749106.0,2347.0,2691358.0,1.0,225.0,...,262062.0,4860175.0,NaN,0.250000,0.080000,NaN,0.150000,NaN,NaN,0.150000
2022-03-13,NaN,2253986.0,8910748.0,8801915.0,8447093.0,5753385.0,2347.0,2691361.0,1.0,293.0,...,260223.0,4844986.0,NaN,0.250000,0.080000,NaN,0.150000,NaN,NaN,0.150000
2022-03-20,NaN,2261034.0,8954306.0,8843195.0,8490545.0,5758267.0,2347.0,2729931.0,1.0,284.0,...,278111.0,4806630.0,NaN,0.392857,0.222857,NaN,0.292857,NaN,NaN,0.292857


In [21]:
fed_bs = data["Balance Sheet: Total Assets ($ Mil)"]

In [22]:
rates = data["Effective Federal Funds Rate"]

In [23]:
fed_bs.set_index("DATE",inplace=True)
rates.set_index("DATE",inplace=True)
rates["INTDSRUSM193N"] = rates[rates["INTDSRUSM193N"] != '.']
rates["INTDSRUSM193N"] = rates["INTDSRUSM193N"].astype(float)
fed_bs.index = pd.to_datetime(fed_bs.index)
rates.index = pd.to_datetime(rates.index)
fed = fed_bs.copy()
fed['Rates'] = rates["INTDSRUSM193N"]
fed = fed.fillna(method="ffill")
fed = fed.dropna()

AttributeError: 'Series' object has no attribute 'set_index'

In [29]:
fed_bs = pd.read_csv("WALCL.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'WALCL.csv'

In [30]:
rates = pd.read_csv("INTDSRUSM193N.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'INTDSRUSM193N.csv'